In [ ]:
import requests
import numpy as np

EURI_API_KEY = ""
def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {EURI_API_KEY}"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

In [9]:
data = """ Making an Impact Helping Millions of Students Succeed
Mentor's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like the mentor himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.
In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, mentor remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world."""

clean_data = data.strip()
max_char = 100
overlap = 20
chunks = []
i = 0
while i < len(clean_data):
    piece = clean_data[i:i+max_char]
    chunks.append(piece)
    i = i+max_char-overlap

In [10]:
emb_list = []
meta = []
for idx, chunk in enumerate(chunks):
    vec = generate_embeddings(chunk)
    print(f"Chunk {idx+1} embedding: {vec}")
    emb_list.append(vec.astype("float32"))
    meta.append({"id":idx, "text":chunk})
xb = np.vstack(emb_list)
d = xb.shape[1]
print(xb.shape)

Chunk 1 embedding: [ 0.01125363 -0.04198496  0.03206514 ... -0.00965171 -0.00748008
 -0.00309157]
Chunk 2 embedding: [ 0.00531732  0.01498619 -0.00798524 ... -0.00841447 -0.00369659
 -0.03715095]
Chunk 3 embedding: [-0.02063813 -0.0040367   0.04868835 ... -0.01943952 -0.00559351
 -0.00585872]
Chunk 4 embedding: [ 0.02577067 -0.00252622  0.08125098 ... -0.0271094  -0.02772728
  0.03045624]
Chunk 5 embedding: [-0.04099081  0.01827518  0.00186117 ... -0.01436205 -0.03385835
 -0.02304866]
Chunk 6 embedding: [ 0.0034383  -0.00736026  0.01752108 ... -0.00409163 -0.04278076
  0.0038498 ]
Chunk 7 embedding: [-0.0210235  -0.06276387  0.02796912 ... -0.0342615   0.03359494
  0.03884748]
Chunk 8 embedding: [ 0.00959746 -0.02052073  0.03461861 ... -0.03847822 -0.00504548
  0.0069495 ]
Chunk 9 embedding: [-0.02211461 -0.01158347  0.02118793 ... -0.0145991  -0.00759404
  0.00940028]
Chunk 10 embedding: [-0.05103014  0.01305801 -0.01491092 ... -0.02556515  0.01165581
  0.02030689]
(10, 1536)


In [ ]:
from qdrant_client import QdrantClient # qdrant tech https://qdrant.tech/

qdrant_client = QdrantClient(
    url="https://85fb270d-b666-4744-8618-a1dccc49b88a.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key="",
)
print(qdrant_client.get_collections())

from qdrant_client import models
collection_name = 'akash_story'  # table name
qdrant_client.recreate_collection( # use latest syntax
    collection_name=collection_name,  # define vector size as 1536 and use cosine search for similarity
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)) ## no need to normalize manually

collections=[CollectionDescription(name='akash_story')]


/tmp/ipykernel_7926/174955392.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection( # use latest syntax


True

In [14]:
points = [] ### perpare the data inside point list
for idx, (chunk, emb) in enumerate(zip(chunks, xb)):
    points.append(
        models.PointStruct(
            id=idx,
            vector=emb.astype("float32").tolist(),
            payload={"text":chunk}
        )
    )
qdrant_client.upsert(collection_name=collection_name, points=points)  ## add records into Qdrant
query = "What is the mission of ineuron?"
q = np.array(generate_embeddings(query)).astype("float32")
qdrant_client.search(collection_name=collection_name, query_vector=q, limit=3, with_payload=True)

/tmp/ipykernel_7926/3278905319.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  qdrant_client.search(collection_name=collection_name, query_vector=q, limit=3, with_payload=True)


[ScoredPoint(id=8, version=0, score=0.48216456, payload={'text': 'ed on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordabl'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2, version=0, score=0.45979592, payload={'text': 'r the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with th'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=5, version=0, score=0.44321448, payload={'text': 'ckgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\nIn 2022,'}, vector=None, shard_key=None, order_value=None)]